# 🔍 LinkedIn Job Scraper & Title Clustering

This project scrapes analyst-related job postings from LinkedIn, extracts job metadata, clusters job titles using embeddings, and stores the results in a SQL Server database.

---

## 📌 Features

- Scrapes LinkedIn jobs by title and location
- Extracts key job information:  
  - Title  
  - Company  
  - Location  
  - Number of applicants  
  - Posting date  
- Encodes and clusters job titles using SentenceTransformers and KMeans
- Assigns standardized job categories to each posting
- Stores and retrieves results in/from Microsoft SQL Server

---

## 🧪 Tech Stack

- **Python**
- **BeautifulSoup** – HTML parsing
- **Pandas** – Data manipulation
- **SQLAlchemy + pyodbc** – SQL Server integration
- **SentenceTransformers** – Text embeddings
- **Scikit-learn** – KMeans clustering

---

## ⚙️ How It Works

### 1. Job Scraping

Jobs are fetched from LinkedIn using public endpoints and parsed using BeautifulSoup.

```python
title = 'Analyst'
location = 'United Kingdom'


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
from urllib.parse import quote_plus


In [2]:
title = 'Analyst'
location='United Kingdom'
title_encoded = quote_plus(title)
location_encoded = quote_plus(location)

In [3]:
start = 0
job_list = []

while True:
    # Build list URL for paginated results
    list_url = (
        f"https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search"
        f"?keywords={title_encoded}&location={location_encoded}&start={start}"
    )

    # Send request to get job listings
    response = requests.get(list_url, headers={"User-Agent": "Mozilla/5.0"})
    list_data = response.text
    list_soup = BeautifulSoup(list_data, "html.parser")

    # Find job listing elements
    jobs = list_soup.find_all("li")
    if not jobs:
        break  # No more jobs found

    id_list = []
    for job in jobs:
        base_card_div = job.find("div", {"class": "base-card"})
        if base_card_div:
            job_id = base_card_div.get("data-entity-urn").split(":")[3]
            print(f"Job ID: {job_id}")
            id_list.append(job_id)

    # Visit job detail page for each job
    for job_id in id_list:
        job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
        job_response = requests.get(job_url, headers={"User-Agent": "Mozilla/5.0"})
        job_soup = BeautifulSoup(job_response.text, "html.parser")

        job_post = {}

        try:
            job_post["title"] = job_soup.find("h2", {
                "class": "top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title"
            }).text.strip()
        except:
            job_post["title"] = None

        try:
            job_post["company_name"] = job_soup.find("a", {
                "class": "topcard__org-name-link topcard__flavor--black-link"
            }).text.strip()
        except:
            job_post["company_name"] = None

        try:
            job_post["location"] = job_soup.find("span", {
                "class": "topcard__flavor topcard__flavor--bullet"
            }).text.strip()
        except:
            job_post["location"] = None

        try:
            job_post["number_of_applicants"] = job_soup.find("figure", {
                "class": "num-applicants__figure topcard__flavor--metadata topcard__flavor--bullet"
            }).text.strip()
        except:
            job_post["number_of_applicants"] = None

        try:
            job_post["posted"] = job_soup.find("span", {
                "class": "posted-time-ago__text topcard__flavor--metadata"
            }).text.strip()
        except:
            job_post["posted"] = None

        if any(job_post.values()):  # At least one field is not None/empty
         job_list.append(job_post)

    print(f"Fetched {len(jobs)} jobs at start={start}")
    start += 10

Job ID: 4205011282
Job ID: 4207664202
Job ID: 4204943056
Job ID: 4200966236
Job ID: 4177801948
Job ID: 4214760140
Job ID: 4207343167
Job ID: 4204334816
Job ID: 4202997714
Job ID: 4209116448
Fetched 10 jobs at start=0
Job ID: 4209116448
Job ID: 4205712236
Job ID: 4214779212
Job ID: 4204801212
Job ID: 4202802795
Job ID: 4175856135
Job ID: 4214696295
Job ID: 4206472637
Job ID: 4204361644
Job ID: 4201279269
Fetched 10 jobs at start=10


In [4]:
df= pd.DataFrame(job_list)
df

,title,company_name,location,number_of_applicants,posted
0,Retail Analyst,the LEGO Group,"London, England, United Kingdom",None,2 weeks ago
1,Equity Analyst - $2bn AUM Long-Biased Hedge Fund,Mondrian Alpha,"Greater London, England, United Kingdom",Over 200 applicants,1 week ago
2,Market Research Analyst,Sony Interactive Entertainment,"London, England, United Kingdom",None,2 weeks ago
3,Junior Business Analyst (Regulatory Reporting),Capgemini,"London Area, United Kingdom",Over 200 applicants,3 weeks ago
4,Business Operations | Analyst/Associate | London,Goldman Sachs,"London, England, United Kingdom",Over 200 applicants,1 week ago
5,Business Insights Analyst,the LEGO Group,"Slough, England, United Kingdom",Over 200 applicants,4 days ago
6,Data Analyst,Aston Martin F1 Team,"Silverstone, England, United Kingdom",Over 200 applicants,2 weeks ago
7,Jr. Procurement Business Analyst,Jefferies,"London Area, United Kingdom",Over 200 applicants,3 weeks ago
8,Business Data Analyst,Capgemini,"London Area, United Kingdom",Over 200 applicants,3 weeks ago
9,"Risk Analyst, UK",Crypto.com,"London, England, United Kingdom",None,1 week ago


### 2.Data Storage
Scraped jobs are inserted into a SQL Server database (SampleDB) via SQLAlchemy:

In [5]:
from sqlalchemy import create_engine,text
import pyodbc

In [7]:
server = "localhost\\SQLEXPRESS"
database = "SampleDB"
driver = "ODBC Driver 17 for SQL Server"
engine = create_engine(
    f"mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver={driver}"
)
con = engine.connect()

In [8]:
df.to_sql('LinkedIN_jobs', con=engine, if_exists='append', index=False)



20

### 3. Job Title Clustering
Job titles are encoded using a sentence embedding model:



In [14]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(df['title'].tolist())

kmeans = KMeans(n_clusters=10, random_state=42)
df['title_cluster'] = kmeans.fit_predict(embeddings)


In [15]:
# Group by cluster and view some example titles
for cluster in sorted(df['title_cluster'].unique()):
    print(f"\n--- Cluster {cluster} ---")
    print(df[df['title_cluster'] == cluster]['title'].head(5).tolist())



--- Cluster 0 ---
['- Data Analyst - CONTRACT', 'Aerodynamic Data Analyst', 'Associate Data Analyst - Energy and Freight Markets', 'Business Data Analyst', 'Business Data Analyst']

--- Cluster 1 ---
['2025 Full-Time Analyst Program - EMEA', '2025 Full-Time Analyst Program - EMEA', 'Analyst', 'Analyst - Industrials', 'Analyst - Industrials']

--- Cluster 2 ---
['Credit Middle office Analyst - Up to £65k + Bonus + Package - London/ Hybrid working', 'Credit Middle office Analyst - Up to £65k + Bonus + Package - London/ Hybrid working', 'FP&A Analyst – Real Estate – Developments/Construction - £60k + Bonus', 'Graduate Business Modelling Analyst - London - Up to £40,000 + amazing bonus/benefits package', 'Investment Data Analyst – Hedge Fund - £65-100k base + bonus']

--- Cluster 3 ---
['Equity Analyst - $2bn AUM Long-Biased Hedge Fund', 'Equity Analyst - $2bn AUM Long-Biased Hedge Fund', 'Equity Analyst - $2bn AUM Long-Biased Hedge Fund', 'Hedge Fund Credit Analyst', 'L/S Equity Analyst 

### 4. Labeling Clusters
Clusters are manually mapped to standardized analyst categories:

In [16]:
cluster_labels = {
    0: "Data Analyst / Quantitative Analyst",
    1: "Generalist Analyst Programs / Graduate Roles",
    2: "Financial Planning / Credit / Business Modeling Analyst",
    3: "Equity / Hedge Fund Analyst",
    4: "Business Analyst / Associate Roles",
    5: "Private Credit / Climate / Infrastructure PE Analyst",
    6: "Private Equity Analyst",
    7: "Investment Banking / Corporate Finance Analyst",
    8: "Compliance / KYC / AML Analyst",
    9: "Trading / Credit Risk Analyst"
}



df['standard_title'] = df['title_cluster'].map(cluster_labels)


In [17]:
server = "localhost\\SQLEXPRESS"
database = "SampleDB"
driver = "ODBC Driver 17 for SQL Server"

# Use an f-string to inject variables into the connection string
cnxn = pyodbc.connect(
    f'DRIVER={{{driver}}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
)

cursor = cnxn.cursor()
cursor.execute("SELECT distinct * FROM LinkedIN_jobs")
columns = [column[0] for column in cursor.description]
rows = [tuple(row) for row in cursor.fetchall()] 

# Create DataFrame with column names
df = pd.DataFrame(rows, columns=columns)



### 5. Final Output
The data, now enriched with standard_title, is saved back to SQL Server:


In [18]:
df.to_sql('LinkedIN_jobs', con=engine, if_exists='replace', index=False)

42

---

## 📂 Tables Used

- **LinkedIN_jobs** – Stores enriched job postings including title, company, location, applicants, post date, cluster label, and standardized job title.

---

## 🚀 Future Improvements

- Add support for more job functions and geographic regions
- Implement database schema versioning (e.g., Alembic)
- Automate daily scraping and clustering
- Integrate a dashboard or reporting layer (e.g., Streamlit, Power BI)

---

## 📜 Disclaimer

This project uses publicly accessible LinkedIn endpoints for educational and research purposes only.  
Please use it responsibly and ensure compliance with [LinkedIn’s Terms of Service](https://www.linkedin.com/legal/user-agreement).
